# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

#### Importing and Starting spark session

In [7]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
spark = SparkSession.builder.appName('First_LinearRegression_Project').getOrCreate()

#### Loading the cruise_ship_info csv file and printing the schema.

In [9]:
shipinfo = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)
shipinfo.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



#### Displaying data from the Dataframe

In [10]:
shipinfo.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

#### To predict crew members we are grouping the required columns

In [11]:
calculator = VectorAssembler(inputCols=["Age", "Tonnage", "passengers","length","cabins","passenger_density"],
                             outputCol="features")

In [12]:
output = calculator.transform(shipinfo)

In [13]:
total_data = output.select("features",'crew')

In [14]:
train_data,test_data = total_data.randomSplit([0.7,0.3])

In [15]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               103|
|   mean|  7.31893203883496|
| stddev|3.4171852057841576|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [16]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                55|
|   mean|  8.68418181818182|
| stddev|3.5202707655273144|
|    min|               1.6|
|    max|              21.0|
+-------+------------------+



In [17]:
lr = LinearRegression(labelCol='crew')

#### Training the LR model

In [18]:
lrModel = lr.fit(train_data,)

In [19]:
test_results = lrModel.evaluate(test_data)

In [20]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.8694074618187599|
| -1.3969277491852523|
|  -0.733698209202025|
| -0.2571051385850964|
|  0.3725306354375366|
|-0.25052666310989125|
| 0.26011788835616656|
|-0.45455697046964794|
| -0.5216485390338299|
|  0.5228754381996978|
|-0.08524489397008139|
| 0.18197160053618422|
| -0.5023858257282487|
| -0.6108527734402518|
|  1.6988899778408584|
| -0.9543243506768277|
| -0.4378113862313242|
|  0.9302986264103925|
| -0.6971265224910379|
|   7.068655248090755|
+--------------------+
only showing top 20 rows



In [21]:
unlabeled_data = test_data.select('features')

In [22]:
predictions = lrModel.transform(unlabeled_data)

In [23]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...| 20.13059253818124|
|[5.0,86.0,21.04,9...| 9.396927749185252|
|[6.0,90.0,20.0,9....| 9.733698209202025|
|[6.0,112.0,38.0,9...|11.157105138585097|
|[6.0,113.0,37.82,...|11.627469364562463|
|[6.0,158.0,43.7,1...|13.850526663109891|
|[8.0,77.499,19.5,...| 8.739882111643833|
|[8.0,110.0,29.74,...|12.054556970469648|
|[9.0,85.0,19.68,9...|  9.21164853903383|
|[9.0,88.5,21.24,9...| 9.777124561800303|
|[9.0,116.0,26.0,9...|11.085244893970081|
|[10.0,68.0,10.8,7...| 6.178028399463816|
|[10.0,86.0,21.14,...| 9.702385825728248|
|[10.0,90.09,25.01...| 9.190852773440252|
|[10.0,151.4,26.2,...|10.831110022159141|
|[11.0,85.0,18.48,...| 8.954324350676828|
|[11.0,86.0,21.24,...| 9.737811386231325|
|[11.0,90.0,22.4,9...|10.069701373589607|
|[11.0,90.09,25.01...| 9.177126522491038|
|[11.0,110.0,29.74...|12.031344751909247|
+--------------------+------------

In [24]:
test_data.show()

+--------------------+-----+
|            features| crew|
+--------------------+-----+
|[4.0,220.0,54.0,1...| 21.0|
|[5.0,86.0,21.04,9...|  8.0|
|[6.0,90.0,20.0,9....|  9.0|
|[6.0,112.0,38.0,9...| 10.9|
|[6.0,113.0,37.82,...| 12.0|
|[6.0,158.0,43.7,1...| 13.6|
|[8.0,77.499,19.5,...|  9.0|
|[8.0,110.0,29.74,...| 11.6|
|[9.0,85.0,19.68,9...| 8.69|
|[9.0,88.5,21.24,9...| 10.3|
|[9.0,116.0,26.0,9...| 11.0|
|[10.0,68.0,10.8,7...| 6.36|
|[10.0,86.0,21.14,...|  9.2|
|[10.0,90.09,25.01...| 8.58|
|[10.0,151.4,26.2,...|12.53|
|[11.0,85.0,18.48,...|  8.0|
|[11.0,86.0,21.24,...|  9.3|
|[11.0,90.0,22.4,9...| 11.0|
|[11.0,90.09,25.01...| 8.48|
|[11.0,110.0,29.74...| 19.1|
+--------------------+-----+
only showing top 20 rows



####  From the above analysis, we can see that our prediction is close to the actual value in most of the cases.